In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm 
import sys
sys.path.insert(1, '../features')

In [2]:
import editdistance

In [3]:
train = pd.read_csv("../dataset/expanded/train_xgb.csv")
train

,queried_record_id,predicted_record_id,cosine_score,linked_id_idx
0,10000002,10000002,1.000000,1
1,10000002,10153548,0.968726,183353
2,10000002,10094260,0.967284,112195
3,10000002,10068162,0.962878,81180
4,10000002,10124826,0.961480,149058
5,10000002,11002952,0.960928,259232
6,10000002,10070393,0.959230,83788
7,10000002,10053080,0.958790,63292
8,10000002,10208430,0.958662,248474
9,10000002,10199695,0.957518,238028


In [4]:
# Add features to train

In [5]:
def target(df_exp_train, df_val):
    df_exp_train = df_exp_train.merge(df_val[['record_id', 'linked_id']], how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)

    def extract_target(predicted, linked):
        res = np.where(predicted == linked, 1, 0)
        return res

    df_exp_train['target'] = extract_target(df_exp_train.predicted_record_id.values, df_exp_train.linked_id.values)
    return df_exp_train

In [6]:
df_val = pd.read_csv("../dataset/validation/test.csv", escapechar="\\")
df_val

,record_id,name,type,address,phone,email,modification,linked_id
0,10127893-T1,JUNO STAR INVESTMENT LIMITED,entity,NaN,1.953617e+11,colpulting@icaoud.cz,typo,10127893
1,12187326-M1,"HUNG, YU-CHI",officer,No.35 Aly.28 Lane43 Sec.3 Ta Feng Rd.; Tanzi D...,NaN,NaN,missing,12187326
2,10196371-T0,BROADOAK INVESTMENTS LIdITED,entity,NaN,8.154367e+12,NaN,typo,10196371
3,10102093-NV2,SANDCASTLE TRADING LTD,entity,NaN,4.996521e+10,info@hotmail.com,new_value,10102093
4,10025744,"Ontime Electronic Technology Co., Ltd",entity,NaN,NaN,NaN,original,10025744
5,10109220,Trehearne Limited,entity,NaN,NaN,NaN,original,10109220
6,10114863,EASTWOOD HOLDINGS DEVELOPMENT LTD.,entity,NaN,1.850728e+10,NaN,original,10114863
7,10040402-T3,MOUNTAIN PROPERTIES S.A,entity,NaN,4.209243e+11,NaN,typo,10040402
8,12153582,Ivone Riesna Zakaria,officer,D.H.I BARAT 9/9 BLOK-A/301-302 RT/RW 009/005 S...,3.923573e+09,IvoneRiesnaZakaria@hotmail.it,original,12153582
9,12096823-T3,ITAY BARAK,officer,12 Bney Dror St.; Bney Dror; Israel,3.362592e+11,NaN,typo,12096823


In [7]:
train = target(train, df_val)

In [8]:
def compute_editdistance(df_exp, validation=True):
    if validation:
        df_test = pd.read_csv("../dataset/validation/test.csv", escapechar="\\")
        df_train = pd.read_csv("../dataset/validation/train.csv", escapechar="\\")
    else:
        df_test = pd.read_csv("../dataset/original/test.csv", escapechar="\\")
        df_train = pd.read_csv("../dataset/original/train.csv", escapechar="\\")

    df_train = df_train.sort_values(by='record_id').reset_index(drop=True)
    df_test = df_test.sort_values(by='record_id').reset_index(drop=True)

    df_exp = df_exp.merge(df_test[['record_id', 'name']], how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)
    df_exp = df_exp.rename(columns={'name': 'queried_name'})

    df_exp = df_exp.merge(df_train[['record_id', 'name']], how='left', left_on='linked_id_idx', right_index=True).drop('record_id', axis=1)
    df_exp = df_exp.rename(columns={'name': 'predicted_name'})

    print(f'NaN on queried_name: {df_exp.queried_name.isna().sum()}')
    print(f'Nan on predicted_name: {df_exp.predicted_name.isna().sum()}')
    
    df_exp['queried_name'] = df_exp.queried_name.fillna('')
    df_exp['predicted_name'] = df_exp.predicted_name.fillna('')

    df_exp['queried_name'] = df_exp.queried_name.str.lower()
    df_exp['predicted_name'] = df_exp.predicted_name.str.lower()

    def extract_editdistance(queried_name, predicted_name):
        res = np.empty(len(queried_name), dtype=int)
        for i in tqdm(range(len(queried_name))):
            try:
                res[i] = editdistance.eval(queried_name[i], predicted_name[i])
            except:
                print(i)
        return res

    df_exp['editdistance'] = extract_editdistance(df_exp.queried_name.values, df_exp.predicted_name.values)
    df_exp = df_exp.drop(['queried_name', 'predicted_name'], axis=1)
    return df_exp['editdistance']

In [9]:
train['editdistance'] = compute_editdistance(train, validation=True)

NaN on queried_name: 30
Nan on predicted_name: 35


In [10]:
email_pop = pd.read_csv("../dataset/validation/feature/email_popularity.csv")
linked_id_pop = pd.read_csv("../dataset/validation/feature/linked_id_popularity.csv")
name_pop = pd.read_csv("../dataset/validation/feature/name_popularity.csv")
nonnull_addr = pd.read_csv("../dataset/validation/feature/number_of_non_null_address.csv")
nonnull_email = pd.read_csv("../dataset/validation/feature/number_of_non_null_email.csv")
nonnull_phone = pd.read_csv("../dataset/validation/feature/number_of_non_null_phone.csv")
phone_pop = pd.read_csv("../dataset/validation/feature/phone_popularity.csv")
name_length = pd.read_csv("../dataset/validation/feature/test_name_length.csv")

In [11]:
train = train.merge(email_pop, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)
train 

,queried_record_id,predicted_record_id,cosine_score,linked_id_idx,linked_id,target,editdistance,email_popularity
0,10000002,10000002,1.000000,1,10000002,1,0,154
1,10000002,10153548,0.968726,183353,10000002,0,21,154
2,10000002,10094260,0.967284,112195,10000002,0,22,154
3,10000002,10068162,0.962878,81180,10000002,0,23,154
4,10000002,10124826,0.961480,149058,10000002,0,32,154
5,10000002,11002952,0.960928,259232,10000002,0,33,154
6,10000002,10070393,0.959230,83788,10000002,0,32,154
7,10000002,10053080,0.958790,63292,10000002,0,32,154
8,10000002,10208430,0.958662,248474,10000002,0,25,154
9,10000002,10199695,0.957518,238028,10000002,0,29,154


In [12]:
train = train.merge(linked_id_pop, how='left', left_on='predicted_record_id', right_on='linked_id', suffixes=['' ,'_y']).drop('linked_id_y', axis=1).rename(columns={'popularity':'linked_id_popularity'})
train

,queried_record_id,predicted_record_id,cosine_score,linked_id_idx,linked_id,target,editdistance,email_popularity,linked_id_popularity
0,10000002,10000002,1.000000,1,10000002,1,0,154,1
1,10000002,10153548,0.968726,183353,10000002,0,21,154,4
2,10000002,10094260,0.967284,112195,10000002,0,22,154,2
3,10000002,10068162,0.962878,81180,10000002,0,23,154,3
4,10000002,10124826,0.961480,149058,10000002,0,32,154,1
5,10000002,11002952,0.960928,259232,10000002,0,33,154,1
6,10000002,10070393,0.959230,83788,10000002,0,32,154,3
7,10000002,10053080,0.958790,63292,10000002,0,32,154,2
8,10000002,10208430,0.958662,248474,10000002,0,25,154,1
9,10000002,10199695,0.957518,238028,10000002,0,29,154,1


In [13]:
train = train.merge(name_pop, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)

In [14]:
train = train.merge(nonnull_addr, how='left', left_on='predicted_record_id', right_on='linked_id', suffixes=['' ,'_y']).drop('linked_id_y', axis=1)

In [17]:
train = train.merge(nonnull_email, how='left', left_on='predicted_record_id', right_on='linked_id', suffixes=['' ,'_y']).drop('linked_id_y', axis=1)
train = train.merge(nonnull_phone, how='left', left_on='predicted_record_id', right_on='linked_id', suffixes=['' ,'_y']).drop('linked_id_y', axis=1)

In [18]:
train = train.merge(phone_pop, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)

In [19]:
train = train.merge(name_length, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)

In [20]:
train = train.fillna(0)

In [21]:
train['linked_id_popularity'] = train.linked_id_popularity.astype(int)
train['null_address'] = train.null_address.astype(int)
train['null_email'] = train.null_email.astype(int)
train['null_phone'] = train.null_phone.astype(int)
train

,queried_record_id,predicted_record_id,cosine_score,linked_id_idx,linked_id,target,editdistance,email_popularity,linked_id_popularity,name_popularity,null_address,perc_non_null_address,null_email,perc_non_null_email,null_phone,perc_non_null_phone,phone_popularity,test_name_length
0,10000002,10000002,1.000000,1,10000002,1,0,154,1,3,1,0,1,0,1,0,2,43
1,10000002,10153548,0.968726,183353,10000002,0,21,154,4,3,4,0,1,75,0,100,2,43
2,10000002,10094260,0.967284,112195,10000002,0,22,154,2,3,2,0,1,50,1,50,2,43
3,10000002,10068162,0.962878,81180,10000002,0,23,154,3,3,3,0,3,0,3,0,2,43
4,10000002,10124826,0.961480,149058,10000002,0,32,154,1,3,1,0,1,0,1,0,2,43
5,10000002,11002952,0.960928,259232,10000002,0,33,154,1,3,1,0,0,100,0,100,2,43
6,10000002,10070393,0.959230,83788,10000002,0,32,154,3,3,3,0,3,0,3,0,2,43
7,10000002,10053080,0.958790,63292,10000002,0,32,154,2,3,2,0,0,100,0,100,2,43
8,10000002,10208430,0.958662,248474,10000002,0,25,154,1,3,1,0,0,100,0,100,2,43
9,10000002,10199695,0.957518,238028,10000002,0,29,154,1,3,1,0,0,100,0,100,2,43


# XGBoost

In [33]:
import xgboost as xgb

group = train.groupby('queried_record_id').size().values
ranker = xgb.XGBRanker()
ranker.fit(train.drop(['queried_record_id', 'target', 'linked_id_idx', 'linked_id'], axis=1), train['target'], group=group)

XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
          max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
          n_jobs=-1, nthread=None, objective='rank:pairwise', random_state=0,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
          subsample=1)

# Test

In [23]:
test = pd.read_csv("../dataset/expanded/test_xgb.csv")

In [24]:
test['editdistance'] = compute_editdistance(test, validation=False)

NaN on queried_name: 10
Nan on predicted_name: 14


In [25]:
email_pop = pd.read_csv("../dataset/original/feature/email_popularity.csv")
linked_id_pop = pd.read_csv("../dataset/original/feature/linked_id_popularity.csv")
name_pop = pd.read_csv("../dataset/original/feature/name_popularity.csv")
nonnull_addr = pd.read_csv("../dataset/original/feature/number_of_non_null_address.csv")
nonnull_email = pd.read_csv("../dataset/original/feature/number_of_non_null_email.csv")
nonnull_phone = pd.read_csv("../dataset/original/feature/number_of_non_null_phone.csv")
phone_pop = pd.read_csv("../dataset/original/feature/phone_popularity.csv")
name_length = pd.read_csv("../dataset/original/feature/test_name_length.csv")

In [26]:
test = test.merge(email_pop, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)

In [28]:
test = test.merge(linked_id_pop, how='left', left_on='predicted_record_id', right_on='linked_id').drop('linked_id', axis=1).rename(columns={'popularity':'linked_id_popularity'})
test

,queried_record_id,predicted_record_id,cosine_score,linked_id_idx,editdistance,email_popularity,linked_id_popularity
0,10000003-TST-MR,10097822,0.913043,174035,12,304,5
1,10000003-TST-MR,10022487,0.903562,40456,12,304,1
2,10000003-TST-MR,10090422,0.898807,160717,9,304,4
3,10000003-TST-MR,10003736,0.897201,6781,16,304,3
4,10000003-TST-MR,10002385,0.895827,4317,18,304,2
5,10000003-TST-MR,10113802,0.891593,202803,18,304,1
6,10000003-TST-MR,10077754,0.891569,137852,16,304,6
7,10000003-TST-MR,10010930,0.891569,19656,14,304,6
8,10000003-TST-MR,10205293,0.891268,365766,12,304,3
9,10000003-TST-MR,10093582,0.889108,166392,10,304,4


In [29]:
test = test.merge(name_pop, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)
test = test.merge(nonnull_addr, how='left', left_on='predicted_record_id', right_on='linked_id').drop('linked_id', axis=1)
test = test.merge(nonnull_email, how='left', left_on='predicted_record_id', right_on='linked_id').drop('linked_id', axis=1)
test = test.merge(nonnull_phone, how='left', left_on='predicted_record_id', right_on='linked_id').drop('linked_id', axis=1)
test = test.merge(phone_pop, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)
test = test.merge(name_length, how='left', left_on='queried_record_id', right_on='record_id').drop('record_id', axis=1)
test = test.fillna(0)

In [30]:
test

,queried_record_id,predicted_record_id,cosine_score,linked_id_idx,editdistance,email_popularity,linked_id_popularity,name_popularity,null_address,perc_non_null_address,null_email,perc_non_null_email,null_phone,perc_non_null_phone,phone_popularity,test_name_length
0,10000003-TST-MR,10097822,0.913043,174035,12,304,5,1,5,0,5,0,5,0,1,18
1,10000003-TST-MR,10022487,0.903562,40456,12,304,1,1,1,0,1,0,0,100,1,18
2,10000003-TST-MR,10090422,0.898807,160717,9,304,4,1,3,25,4,0,2,50,1,18
3,10000003-TST-MR,10003736,0.897201,6781,16,304,3,1,3,0,3,0,0,100,1,18
4,10000003-TST-MR,10002385,0.895827,4317,18,304,2,1,1,50,2,0,2,0,1,18
5,10000003-TST-MR,10113802,0.891593,202803,18,304,1,1,1,0,0,100,0,100,1,18
6,10000003-TST-MR,10077754,0.891569,137852,16,304,6,1,6,0,5,16,3,50,1,18
7,10000003-TST-MR,10010930,0.891569,19656,14,304,6,1,4,33,6,0,0,100,1,18
8,10000003-TST-MR,10205293,0.891268,365766,12,304,3,1,3,0,3,0,2,33,1,18
9,10000003-TST-MR,10093582,0.889108,166392,10,304,4,1,2,50,4,0,0,100,1,18


In [31]:
test['linked_id_popularity'] = test.linked_id_popularity.astype(int)
test['null_address'] = test.null_address.astype(int)
test['null_email'] = test.null_email.astype(int)
test['null_phone'] = test.null_phone.astype(int)

In [38]:
predictions = ranker.predict(test.drop(['queried_record_id', 'linked_id_idx'], axis=1))
test['predictions'] = predictions
df_predictions = test[['queried_record_id', 'predicted_record_id', 'predictions']]

In [41]:
rec_pred = []
for (r,p) in zip(df_predictions.predicted_record_id, df_predictions.predictions):
    rec_pred.append((r, p))
rec_pred

[(10097822, -4.64740514755249),
 (10022487, -4.818333148956299),
 (10090422, -4.64740514755249),
 (10003736, -4.64740514755249),
 (10002385, -4.64740514755249),
 (10113802, -4.818333148956299),
 (10077754, -4.64740514755249),
 (10010930, -4.64740514755249),
 (10205293, -4.64740514755249),
 (10093582, -4.64740514755249),
 (10043172, 2.0654613971710205),
 (10211873, -4.890035629272461),
 (10080769, -4.890035629272461),
 (10186437, -4.890035629272461),
 (10029584, -4.890035629272461),
 (10107250, -4.890035629272461),
 (10206650, -4.890035629272461),
 (10007755, -4.890035629272461),
 (10128448, -4.890035629272461),
 (10079344, -4.890035629272461),
 (10000010, 3.0790200233459473),
 (10198226, -3.9924511909484863),
 (10101221, -4.890035629272461),
 (10021610, -3.9924511909484863),
 (10108279, -3.9924511909484863),
 (10124067, -3.9924511909484863),
 (10186594, -4.890035629272461),
 (10057797, -1.625673770904541),
 (10005319, -1.520430088043213),
 (10214153, -4.890035629272461),
 (10153043, -1

In [42]:
df_predictions['rec_pred'] = rec_pred
group_queried = df_predictions[['queried_record_id', 'rec_pred']].groupby('queried_record_id').apply(lambda x: list(x['rec_pred']))
df_predictions = pd.DataFrame(group_queried).reset_index().rename(columns={0 : 'rec_pred'})

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
def reorder_preds(preds):
    sorted_list = []
    for i in range(len(preds)):
        l = sorted(preds[i], key=lambda t: t[1], reverse=True)
        l = [x[0] for x in l]
        sorted_list.append(l)
    return sorted_list

In [44]:
df_predictions['ordered_preds'] = reorder_preds(df_predictions.rec_pred.values)
df_predictions = df_predictions[['queried_record_id', 'ordered_preds']].rename(columns={'ordered_preds': 'predicted_record_id'})

In [45]:
new_col = []
for t in tqdm(df_predictions.predicted_record_id):
    new_col.append(' '.join([str(x) for x in t]))
new_col

['10097822 10090422 10003736 10002385 10077754 10010930 10205293 10093582 10022487 10113802',
 '10043172 10211873 10080769 10186437 10029584 10107250 10206650 10007755 10128448 10079344',
 '10000010 10005319 10057797 10198226 10021610 10108279 10124067 10101221 10186594 10214153',
 '10153043 10147826 10071664 12136105 10189807 10107777 10027275 10139664 10127126 10202426',
 '10147072 10012410 10002588 10133364 10002937 10113057 10186211 10034520 10006940 10134861',
 '10196127 10170427 12131717 10152573 10209898 10087602 10079062 10092259 10126538 10125867',
 '10000018 10000069 12180349 10014054 10100274 10160621 12115160 12188120 10187290 10075075',
 '10000018 10000069 12180349 10014054 10100274 10160621 12115160 12188120 10187290 10075075',
 '10000020 10094948 10140964 10070569 10113164 10062721 10173393 10128438 10099208 10127211',
 '10000020 10094948 10140964 10070569 10113164 10062721 10173393 10128438 10099208 10127211',
 '10000022 10213858 10185742 10199378 10146465 10056965 1100

In [46]:
df_predictions.predicted_record_id = new_col

In [47]:
df_predictions.to_csv('xgb_sub4.csv', index=False)

In [48]:
df_predictions.shape

(266953, 2)

In [49]:
sub_old = pd.read_csv("/Users/alessiorussointroito/Documents/GitHub/Oracle_HPC_contest/notebooks/xgb_sub.csv")

In [51]:
set(sub_old.queried_record_id.values) - set(df_predictions.queried_record_id.values)

{'12026587-TST-MR', '13009531-TST-MR'}

In [52]:
missing_values = {'queried_record_id' : ['12026587-TST-MR', '13009531-TST-MR'], 
                 'predicted_record_id': [10111147, 10111147]}
missing_df = pd.DataFrame(missing_values)
missing_df

,queried_record_id,predicted_record_id
0,12026587-TST-MR,10111147
1,13009531-TST-MR,10111147


In [53]:
df_predictions = pd.concat([df_predictions, missing_df])

In [54]:
df_predictions.to_csv('xgb_sub3.csv', index=False)

In [55]:
train.target.sum() 

166938

In [57]:
train.queried_record_id.shape[0] / 10

228174.0